<a href="https://colab.research.google.com/github/FALABELLA-GFSNEYDER/modeloprediccion-colab-gcp/blob/main/SubirGCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import zipfile
from pyspark.ml import PipelineModel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=521534be87325f8b854e84eef4c1c632705856889c2369bca1e6ec6da31b35bd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autenticación
auth.authenticate_user()

# Proyecto y cliente de BigQuery
project_id = 'bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Consulta
query = f"""
    SELECT *
    FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.tallas_prediccion`
"""

# Cargar el DataFrame desde BigQuery
df_espejo = client.query(query).to_dataframe()

#Ver la data
df_espejo.head()

,sellerName,product_name,shop_sku,primary_category,N1,width_in_cm,length_in_cm,height_in_cm,weight_in_kg,volumen_cm3,size
0,COMERCIAL PAOLO,inodoro one pice blanco loza vitrificada,124000327,wc,Construccion y ferreteria,45.0,70.0,60.0,40.00,189000.0,XL
1,VAINSA,inodoro suspendido redondo blanco vainsa,118064433,wc,Construccion y ferreteria,42.0,55.0,120.0,43.70,277200.0,XL
2,TEKA KUCHENTECHNIK PERU,inodoro compacto formentera,119576862,wc,Construccion y ferreteria,69.0,37.0,90.0,53.00,229770.0,XXL
3,VAINSA,one piece manhattan blanco vainsa,121828579,wc,Construccion y ferreteria,74.0,85.0,44.0,60.10,276760.0,XXL
4,VAINSA,one piece sestri blanco vainsa,121827440,wc,Construccion y ferreteria,45.5,76.0,79.5,41.85,274911.0,XL


In [ ]:
#Extraerel zip
# Nombre del archivo zip que subiste
zip_filename = 'SPM_spark.zip'

# Descomprimir el archivo
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('SPM_spark')

In [ ]:
#Cargar el modelo
# Cargar el modelo desde el directorio descomprimido
loaded_model = PipelineModel.load("/content/SPM_spark/content/SPM_spark")

# Ahora puedes utilizar el modelo cargado en Colab

In [ ]:
from pyspark.sql import SparkSession
# Inicializar la sesión de Spark
spark = SparkSession.builder.master("local").appName("SPM-SPARK").getOrCreate()

In [ ]:
# Definir la función UDF para lematizar texto
def lemmatize_text_udf(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Definir las funciones UDF para tokenizar y limpiar texto
def tokenize_and_clean_udf(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in spanish_stop_words]
    return tokens